In [110]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [111]:
df = pd.read_csv('https://github.com/murpi/wilddata/raw/master/quests/tweets.zip')
df.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [112]:
# ne garder que les tweet positifs et négatifs
df = df[df['sentiment'].isin(['negative', 'positive'])]

In [113]:
# pourcentage de tweets positifs et négatifs
df.sentiment.value_counts(normalize=True)*100

positive    52.447595
negative    47.552405
Name: sentiment, dtype: float64

52.44 % de tweet positifs / 47.55% de tweet négatifs

In [114]:
# X,y et train_test_split
X = df.text
y = df.sentiment

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 32)


In [115]:
# Crée un modèle vectorizer
vectorizer = CountVectorizer()

# Entraîne ton modèle sur X_train
vectorizer.fit(X_train)

# créer sparse_matrix sur X_train
X_train_CV = vectorizer.transform(X_train)

# créer sparse_matrix sur X_test SANS FIT
X_test_CV = vectorizer.transform(X_test)

# format matrix de test
X_test_CV

<4091x15806 sparse matrix of type '<class 'numpy.int64'>'
	with 44633 stored elements in Compressed Sparse Row format>

In [116]:
# Regression Logistique
model = LogisticRegression().fit(X_train_CV,y_train)

print('Accuracy score on the train set :', model.score(X_train_CV, y_train))
print('Accuracy score on the test set :', model.score(X_test_CV, y_test))  

Accuracy score on the train set : 0.9663461538461539
Accuracy score on the test set : 0.8772916157418724


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [117]:
# question bonus : afficher 10 tweet mal prédits

matrix = pd.DataFrame(data=confusion_matrix(y_true = y_test, y_pred = model.predict(X_test_CV)), index=model.classes_ + "actual", columns = model.classes_ + "predicted")

In [118]:
matrix

,negativepredicted,positivepredicted
negativeactual,1700,235
positiveactual,267,1889


In [119]:
new_df = pd.DataFrame(X_test)

# colonne avec prédictions
new_df['predict'] = model.predict(X_test_CV)

# merge avec y_pred pour avoir la colonne sentiment
final_df = pd.merge(new_df, y_test, left_index=True, right_index=True)

final_df.head()

,text,predict,sentiment
5681,"- no, is buttfuck stupid. I`m just silly and...",negative,negative
7662,get better omg i still dont believe that i di...,negative,negative
2671,HollowbabesHere comes the utter shite #bgt <I ...,positive,negative
5021,Thank You Clayton. Going to my favorite Greek...,positive,positive
26963,I`m watching it at the moment -sighs- and st...,negative,negative


In [120]:
final_df[(final_df.predict != final_df.sentiment)].head(10)

,text,predict,sentiment
2671,HollowbabesHere comes the utter shite #bgt <I ...,positive,negative
18732,SUFFICATION NO BREATHING. It`s okay. There`ll...,positive,negative
12055,i wanna vote for Miley Cyrus for the mtv movie...,negative,positive
21824,I love music so much that i`ve gone through pa...,positive,negative
18465,"I can only message those who message me, if we...",negative,positive
2976,"wish I could feel no pain (8) but it`s ok, at...",negative,positive
3922,so glad i`m not at uni anymore,negative,positive
5199,You`re not here. I hope you`re still resting....,negative,positive
468,"you`re missing out, bb! i`m such a cereal nu...",negative,positive
15216,have an amazing time with your mommas tomorro...,positive,negative


In [121]:
final_df['text'][(final_df.predict != final_df.sentiment)].head(10).values

array(['HollowbabesHere comes the utter shite #bgt <I completely agree',
       ' SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times.  *Jinx',
       'i wanna vote for Miley Cyrus for the mtv movie awards..but i don`t know where i could  somebody could send me a link? thaank you <3',
       'I love music so much that i`ve gone through pain to play :S my sides of my fingers now are peeling and have blisters from playing so much',
       'I can only message those who message me, if we`re fwends...so those that want replies..follow me.  hmm..that sounds funny..',
       'wish I could feel no pain (8)  but it`s ok, at least they like Brazil!',
       ' so glad i`m not at uni anymore',
       ' You`re not here. I hope you`re still resting.  I don`t want you to be stressed.',
       '  you`re missing out, bb! i`m such a cereal nut, i think i like every kind available.',
       ' have an amazing time with your mommas tomorrow! Show the

Effectivement l'algo a prédit du négatif quand il y a des négations ou des mots négatifs pourtant le sens de la phrase est plutot positif.